In [2]:
import numpy as np
import pandas as pd
import cobra
import cobra.test
from cobra.test import create_test_model
import os
from os.path import join
from time import time

from cobra.flux_analysis import (
    single_gene_deletion, single_reaction_deletion, double_gene_deletion,
    double_reaction_deletion)

a = cobra.io.read_sbml_model("iEC1344_C.xml")


Academic license - for non-commercial use only - expires 2024-03-07
Using license file /home/schwartzlab/gurobi.lic


In [3]:
a.reactions.EX_xyl__D_e.lower_bound= -10
a.reactions.EX_xyl__D_e.upper_bound= 1000

a.reactions.EX_glc__D_e.lower_bound= 0
a.reactions.EX_glc__D_e.upper_bound= 0


In [4]:
f= 29.900000001
h= round(f, 2)
h

29.9

In [5]:
fmin = -30
fmax = 30
i = -1

while i< 2727:
    x = list()    # x and y need to be reinitialised for each new reaction
    y = list()
    f = fmin      # define a new variable to change the flux, fmin should remain a fixed value

    i = i + 1
    r = a.reactions[i]
    r_id = r.id

    foriginalmax = a.reactions.get_by_id(r_id).upper_bound
    foriginalmin = a.reactions.get_by_id(r_id).lower_bound
    a.reactions.get_by_id(r_id).lower_bound = a.reactions.get_by_id(r_id).upper_bound = round(f, 2)

    while round(f, 2) <= fmax:
        a.reactions.get_by_id(r_id).lower_bound = a.reactions.get_by_id(r_id).upper_bound = round(f, 2)
        
        solution = a.optimize()
        ov_new = solution.objective_value

        x.append(round(f, 2))
        y.append(ov_new)

        f = round(f, 2) + 0.1

    d = pd.DataFrame(x, columns = ['flux'])
    d2 = pd.DataFrame(y, columns = ['Biomass'])
    d['Biomass'] = d2['Biomass']
    biomass_per_flux = d.to_csv('biomass_per_flux_for_reaction_{}_xylose.csv'.format(r_id), index = True)
    a.reactions.get_by_id(r_id).lower_bound = foriginalmin
    a.reactions.get_by_id(r_id).upper_bound = foriginalmax


/home/schwartzlab/miniconda3/lib/python3.8/site-packages/cobra/util/solver.py:508: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


IndexError: list index out of range

In [80]:
a.reactions.get_by_id("34DHALDD").lower_bound = a.reactions.get_by_id("34DHALDD").upper_bound = 0

solution = a.optimize()
solution.objective_value


0.9823963461343016